<a href="https://colab.research.google.com/github/Santosh-Gupta/NaturalLanguageRecommendations/blob/master/notebooks/training/PaperVectorTrainingWord2vec_Github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook was used to create and embedding for each paper using the word2vec algorithm. The 'context' for each paper is the embedding ID for references or citations. The skip-gram version was used, and the number of citations/references were 4 (randomly selected) for each paper. 

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!nvidia-smi

Thu Dec 26 07:00:17 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

import numpy
import random 
import hashlib

import torch.optim as optim
from tqdm import tqdm
import sys
import tensorflow as tf
import collections
import numpy as np

from collections import deque
numpy.random.seed(12345)

use_cuda = True
# if use_cuda and torch.cuda.is_available():
#     net.cuda()

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import collections
import hashlib
import math
import os
import random
import sys
from tempfile import gettempdir
import zipfile
import h5py
import requests
import pickle

# import loguniform
# from loguniform import LogUniform

data_index = 0
vocabulary_size = 14886544+1 

In [0]:
h5file = h5py.File('/content/gdrive/My Drive/NaturalLanguageRecommendations/CS/SemanticScholar1p26M-CS-12-22-19.h5','r')
trainData = np.column_stack(( h5file['paper' ][:] , h5file['cites' ][:] ))


In [0]:
trainData[:,0].max()

1262995

In [0]:
trainData.shape

(1262996, 2)

In [0]:
trainData

array([[235092,
        array([ 931311, 1018430,  523305,  822033,  529623,  938165,  927913],
      dtype=int32)],
       [868823, array([336997], dtype=int32)],
       [350063,
        array([ 153164,  395362,  294290,  189116,  178737,  847601,  528571,
        552113,  901146, 1071567,  439787], dtype=int32)],
       ...,
       [946513, array([  87528, 1252523, 1155882,  204501], dtype=int32)],
       [296967, array([953075, 913188, 307815], dtype=int32)],
       [35043, array([77923], dtype=int32)]], dtype=object)

In [0]:
del h5file
# Save memory

# Create data generator

In [0]:
data_index = 0
epoch_index = 0
recEpoch_indexA = 0 #Used to help keep store of the total number of epoches with the models

def generate_batch(batch_size, inputCount, negRate): #batch size = number of labels
  #inputCount = number of inputs per label
    global data_index, epoch_index
    
    batch = np.ndarray(shape=(batch_size, inputCount), dtype=np.int32) 
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    negs = np.zeros(shape=(batch_size, negRate), dtype=np.int32)
    
    n=0
    while n < batch_size:
        if len( set(trainData[data_index, 1]) ) >= inputCount:
            labels[n,0] = trainData[data_index, 0]
            batch[n] = random.sample( set(trainData[data_index, 1]),  inputCount)
            negs[n] = np.random.randint(vocabulary_size-1, size=(negRate))
        else:
            labels[n,0] = trainData[data_index, 0]
            batch[n] = np.random.choice(trainData[data_index, 1], inputCount)
            negs[n] = np.random.randint(vocabulary_size-1, size=(negRate))

        n = n+1
        data_index = (data_index + 1) % len(trainData) #may have to do something like len my_data[:]
        if data_index == 0:
            epoch_index = epoch_index + 1
            print('Completed %d Epochs' % epoch_index)
    
    return batch, labels, negs    

In [0]:
here, goes, everything = generate_batch(4, 4, 5 ) # to do next, insert %len(headernumber)
print('batch', here)
print('labels', goes)
print('negz', everything)

batch [[ 927913 1018430  822033  931311]
 [ 336997  336997  336997  336997]
 [ 153164  847601 1071567  178737]
 [ 591548  479610  571510  760686]]
labels [[235092]
 [868823]
 [350063]
 [664061]]
negz [[14757349  7735425  1396132  2993577  6164478]
 [ 9523582  5664101  3555874  7708189 13679361]
 [ 4246897  3579195  8002190 12081257 12143707]
 [ 5266167 11294070  6962384  7381874  1585865]]


# Create model architecture

In [0]:
class SkipGramModel(nn.Module):
    """Skip gram model of word2vec.
    Attributes:
        emb_size: Embedding size.
        emb_dimention: Embedding dimention, typically from 50 to 500.
        u_embedding: Embedding for center word.
        v_embedding: Embedding for neibor words.
    """

    def __init__(self, emb_size, emb_dimension):
        """Initialize model parameters.
        Apply for two embedding layers.
        Initialize layer weight
        Args:
            emb_size: Embedding size.
            emb_dimention: Embedding dimention, typically from 50 to 500.
        Returns:
            None
        """
        super(SkipGramModel, self).__init__()
        self.emb_size = emb_size
        self.emb_dimension = emb_dimension
        self.u_embeddings = nn.Embedding(emb_size, emb_dimension, sparse=False).cuda()
        self.v_embeddings = nn.Embedding(emb_size, emb_dimension, sparse=False).cuda()
        # self.targets = torch.ones(34816).cud
        self.init_emb()

    def init_emb(self):
        """Initialize embedding weight like word2vec.
        The u_embedding is a uniform distribution in [-0.5/em_size, 0.5/emb_size], and the elements of v_embedding are zeroes.
        Returns:
            None
        """
        initrange = (2.0 / (vocabulary_size + self.emb_dimension)) ** 0.5 # Xavier init
        self.u_embeddings.weight.data.uniform_(-initrange, initrange)
        self.v_embeddings.weight.data.normal_(mean=0, std=math.sqrt(initrange))
        self.lossFunction = nn.BCEWithLogitsLoss( reduction = 'none' )
        

    def forward(self, pos_u, pos_v, neg_v, targets ):
        """Forward process.
        As pytorch designed, all variables must be batch format, so all input of this method is a list of word id.
        Args:
            pos_u: list of center word ids for positive word pairs.
            pos_v: list of neibor word ids for positive word pairs.
            neg_u: list of center word ids for negative word pairs.
            neg_v: list of neibor word ids for negative word pairs.
        Returns:
            Loss of this process, a pytorch variable.
        """
        emb_u = self.u_embeddings(pos_u)
        emb_v = self.v_embeddings(pos_v)
        neg_emb_v = self.v_embeddings(neg_v)

        scorePos = torch.bmm(emb_u, torch.transpose(emb_v, 1, 2)).squeeze()
        scoreNeg = torch.bmm( emb_u , torch.transpose(neg_emb_v, 1, 2) ).squeeze()

        #reduce to dot products for each set, and concatinate all the losses
        #noticed that the sign change for the negative sample dot products
        totalScores = torch.cat( ( scorePos.reshape(scorePos.shape[0], -1) , -scoreNeg.reshape(scoreNeg.shape[0], -1) ) , dim=1)

        indLoss = self.lossFunction( totalScores, targets )
        rowSum = torch.sum(indLoss, dim=1) #Sum all losses for each set
        finalLoss = torch.mean(rowSum) #Average losses across batches 

        return finalLoss

    def save_embedding(self, id2word, file_name, use_cuda):
        """Save all embeddings to file.
        As this class only record word id, so the map from id to word has to be transfered from outside.
        Args:
            id2word: map from word id to word.
            file_name: file name.
        Returns:
            None.
        """
        if use_cuda:
            embedding = self.u_embeddings.weight.cpu().data.numpy()
        else:
            embedding = self.u_embeddings.weight.data.numpy()
        fout = open(file_name, 'w')
        fout.write('%d %d\n' % (len(id2word), self.emb_dimension))
        for wid, w in id2word.items():
            e = embedding[wid]
            e = ' '.join(map(lambda x: str(x), e))
            fout.write('%s %s\n' % (w, e))


# Optimizer

In [0]:
output_file_name = 'theOutpule.file'
batch_size = 512*2
window_size = 4
negSamp = 132
iterationsMax = 100000 #200001
initial_lr =1.0

num_classes = batch_size
emb_dimension = 512

skip_gram_model = SkipGramModel(num_classes, emb_dimension)
use_cuda = torch.cuda.is_available()

if use_cuda:
    skip_gram_model.cuda()

optimizer = optim.Adagrad(
    skip_gram_model.parameters() )

# We are training with a large number of parameters, which can not all fit within GPU memory, so we are use a library called SpeedTorch (google it) to host some of the parameters on the CPU for fast transfer for CPUs with 2 cores like Colab

In [0]:
!pip install SpeedTorch

In [0]:
import SpeedTorch

In [0]:
trainData[:,0].max()

1262995

In [0]:
# Now with plus 1
vocabulary_size = trainData[:,0].max()+1
emb_dimension = 512

In [0]:
# The total u embeddings stay on the GPU, the total v embeddings stay on the CPU

uEmbed_switcher = SpeedTorch.PytorchModelFactory( skip_gram_model.u_embeddings, total_classes=vocabulary_size, embed_dimension=emb_dimension )
vEmbed_switcher = SpeedTorch.ModelFactory( skip_gram_model.v_embeddings, total_classes=vocabulary_size, embed_dimension=emb_dimension, CPUPinn = True)

In [0]:
uAdagrad_switcher = SpeedTorch.PytorchOptimizerFactory( optimizer, total_classes=vocabulary_size, embed_dimension=emb_dimension, model=skip_gram_model, variable_name= 'u_embeddings')
vAdagrad_switcher = SpeedTorch.OptimizerFactory( optimizer, total_classes=vocabulary_size, embed_dimension=emb_dimension, model=skip_gram_model, variable_name='v_embeddings' , CPUPinn = True)

In [0]:
'''
IF THIS IS YOUR FIRST TIME RUNNING, INIT RANDOMLY. 
IF YOU ALREADY HAVE SAVED WEIGHTS, SEE NEXT CODE BLOCK
'''

# initrange = (2.0 / (10000 + emb_dimension)) ** 0.5 # Xavier init
# uEmbed_switcher.uniformDistributionInit( -initrange, initrange )
# vEmbed_switcher.normalDistributionInit( 0, math.sqrt(initrange) )

'\nIF THIS IS YOUR FIRST TIME RUNNING, INIT RANDOMLY. \nIF YOU ALREADY HAVE SAVED WEIGHTS, SEE NEXT CODE BLOCK\n'

In [0]:
'''
IF YOU HAVE SAVED WEIGHTS, USE THIS CODE BLOCK TO LOAD THEM
'''

uEmbed_switcher.loadTorch('/content/gdrive/My Drive/NaturalLanguageRecommendations/CS/75EpochsfilenameU2.torch')
vEmbed_switcher.loadCupy('/content/gdrive/My Drive/NaturalLanguageRecommendations/CS/75EpochsfilenameV2.cpy.npy')

# vEmbed_switcher.saveCupy('/content/gdrive/My Drive/NaturalLanguageRecommendations/CS/75EpochsfilenameV2.cpy')

# uEmbed_switcher.saveTorch('/content/gdrive/My Drive/NaturalLanguageRecommendations/CS/75EpochsfilenameU2.torch')

In [0]:
uAdagrad_switcher.optInit()
vAdagrad_switcher.optInit()

#Run training

In [0]:
runningLoss = 0
batch_size = 512*2
negSamp = 256
posSamp = 4

labelsTranfored2 = uEmbed_switcher.variableTransformer( batchSize=batch_size, posPerBatch=1)
batchTransformed2, negzTransformed2 = vEmbed_switcher.variableTransformer( batchSize=batch_size, posPerBatch=posSamp, negPerBatch=negSamp )

targets = torch.ones( batch_size, posSamp + negSamp , dtype = torch.float32 ).cuda()

batchTensor = torch.from_numpy(batchTransformed2)
LabelTensor = torch.from_numpy(labelsTranfored2)
negTensor = torch.from_numpy(negzTransformed2)

pos_u = Variable(torch.LongTensor(LabelTensor.long()))
pos_v = Variable(torch.LongTensor(batchTensor.long()))
neg_v = Variable(torch.LongTensor(negTensor.long()))

if use_cuda:
    pos_u = pos_u.cuda()
    pos_v = pos_v.cuda()
    neg_v = neg_v.cuda()

for i in range(650000):

    batch, labels, negz = generate_batch(batch_size=batch_size, inputCount=window_size, negRate= negSamp)

    with torch.no_grad():
        uEmbed_switcher.beforeForwardPass( labels )
        vEmbed_switcher.beforeForwardPass( batch, negz )
        uAdagrad_switcher.beforeForwardPass( labels )
        vAdagrad_switcher.beforeForwardPass( batch, negz )

    optimizer.zero_grad()

    loss = skip_gram_model.forward(pos_u, pos_v, neg_v, targets)
    runningLoss = runningLoss + loss.data.item()

    loss.backward()
    optimizer.step()

    with torch.no_grad():
        uEmbed_switcher.afterOptimizerStep( labels )
        vEmbed_switcher.afterOptimizerStep( batch, negz )
        uAdagrad_switcher.afterOptimizerStep( labels )
        vAdagrad_switcher.afterOptimizerStep( batch, negz )

    if i%500 == 0:
        print('i is ', i)
        print('loss is ', runningLoss/500)
        runningLoss = 0

i is  0
loss is  0.021855310440063478
i is  500
loss is  11.556811813354493
i is  1000
loss is  12.146794439315796
Completed 1 Epochs
i is  1500
loss is  10.651586374282838
i is  2000
loss is  9.705809873580932
Completed 2 Epochs
i is  2500
loss is  9.72007794380188
i is  3000
loss is  8.837360858917236
i is  3500
loss is  9.110045764923095
Completed 3 Epochs
i is  4000
loss is  8.41481383895874
i is  4500
loss is  8.327060220718383
Completed 4 Epochs
i is  5000
loss is  8.30148872756958
i is  5500
loss is  7.651543900489807
i is  6000
loss is  7.906761452674866
Completed 5 Epochs
i is  6500
loss is  7.292380665779114
i is  7000
loss is  7.329499310493469
Completed 6 Epochs
i is  7500
loss is  7.223998176574707
i is  8000
loss is  6.779026105880737
i is  8500
loss is  7.008891167640686
Completed 7 Epochs
i is  9000
loss is  6.4318462152481075
i is  9500
loss is  6.552037642478943
Completed 8 Epochs
i is  10000
loss is  6.401390962600708
i is  10500
loss is  6.103890675544739
i is  1100

KeyboardInterrupt: ignored

# Save embeddings

In [0]:
# save embeddings directly. Warning, the cupy version will take a while, and needs additional ram

vEmbed_switcher.saveCupy('filenameV2.cpy')

uEmbed_switcher.saveTorch('filenameU2.torch')

# Test embeddings

In [0]:
numpyU = uEmbed_switcher.getNumpyVersion()

numpyV = vEmbed_switcher.getNumpyVersion()

In [0]:
#Normalize embeddings

normalizedV = np.divide( numpyV, np.sqrt( np.sum(np.square(numpyV), axis=1) )[:,None] )
normalizedU = np.divide( numpyU, np.sqrt( np.sum(np.square(numpyU), axis=1) )[:,None] )

del numpyU
del numpyV

In [0]:
# combine embeddings

normalizedCombAfter = normalizedU + normalizedV

In [0]:
np.save('/content/gdrive/My Drive/NaturalLanguageRecommendations/CS/CombAfterNormalizaton100Epochs.npy', normalizedCombAfter)

# Get title, embed, absract data to test the embeddings on

In [0]:
import pandas as pd

In [0]:
df = pd.read_json( '/content/gdrive/My Drive/NaturalLanguageRecommendations/CS/TitlesAbstractsEmbedIds.json.gzip', compression = 'gzip' )

In [0]:
embed2Title = pd.Series(df['title'].values,index=df['EmbeddingID']).to_dict()
embed2Title

{235092: 'Simple analytic formulas for PID tuning',
 868823: 'Effect of structure parameters on the losses and efficiency of Surface-Mounted PMSM',
 350063: 'Saturated Throughput Analysis of IEEE 802.11ad EDCA For High Data Rate 5G-IoT Applications',
 664061: 'A Fuzzy Logic Algorithm for Minimizing Error (FLAME) in Wireless Sensor Networks',
 884696: 'FVF based CDTA and Its application',
 969635: 'Speech/music discrimination using awarped LPC-based feature and a fuzzy expert system for intelligent audio coding',
 418364: 'Effective Capacity for Renewal Service Processes With Applications to HARQ Systems',
 531667: 'Mashroom+: An Interactive Data Mashup Approach with Uncertainty Handling',
 1827: 'Robust Filtering for Markov Jump Systems by Randomized Algorithm Approach',
 506588: 'SSENet-2011: A Network Intrusion Detection System dataset and its comparison with KDD CUP 99 dataset',
 918081: 'On the Boundedness of Asymptotic Stability Regions for the Stochastic Theta Method',
 521881: '

# Test similarity properties 

In [0]:
NumberOfPapers = 10

testIndexes = np.array([45773])

TestEmbedsU = normalizedU[testIndexes]
TestEmbedsV = normalizedV[testIndexes]
# TestEmbedsComb = normalizedComb[testIndexes]
TestEmbedsCombAfter = normalizedCombAfter[testIndexes]


simNumpyU = np.matmul(TestEmbedsU, normalizedU.T)
simNumpyV = np.matmul(TestEmbedsV, normalizedV.T)
simNumpyUV = np.matmul(TestEmbedsU, normalizedV.T)
simNumpyVU = np.matmul(TestEmbedsV, normalizedU.T)
# simNumpyComb = np.matmul(TestEmbedsComb, normalizedComb.T) 
simNumpyCombAfter = np.matmul(TestEmbedsCombAfter, normalizedCombAfter.T) 

g1 =  (-simNumpyU).argsort()
h1 = -np.sort(-simNumpyU)[0]
stk1 = np.vstack((g1 , h1) )[: , 0:NumberOfPapers]

g2 =  (-simNumpyV).argsort()
h2 = -np.sort(-simNumpyV)[0]
stk2 = np.vstack((g2 , h2) )[: , 0:NumberOfPapers]

g3 =  (-simNumpyUV).argsort()
h3 = -np.sort(-simNumpyUV)[0]
stk3 = np.vstack((g3 , h3) )[: , 0:NumberOfPapers]

g4 =  (-simNumpyVU).argsort()
h4 = -np.sort(-simNumpyVU)[0]
stk4 = np.vstack((g4 , h4) )[: , 0:NumberOfPapers]

# g5 =  (-simNumpyComb).argsort()
# h5 = -np.sort(-simNumpyComb)[0]
# stk5 = np.vstack((g5 , h5) )[: , 0:NumberOfPapers]

# g6 =  (-simNumpyCombAfter).argsort()
# h6 = -np.sort(-simNumpyCombAfter)[0]
# stk6 = np.vstack((g6 , h6) )[: , 0:NumberOfPapers]

stkAll = np.concatenate((stk1,  stk2, stk3,  stk4), axis = 1)

i = np.argsort(  -stkAll[1]  )
stkAll2 = stkAll[:,i]

_, idx = np.unique(stkAll2[0], return_index=1)
stkAll3 = stkAll2[:, np.sort(idx)]


In [0]:
for i in range(len(testIndexes)):
    paper = embed2Title[testIndexes[i]]
    top_k = 10  # number of nearest neighbors
    nearest = stkAll3[0][1:]
    print('------ Nearest to ', paper, ' -----------------')
    for k in range(top_k):
        close_paper = embed2Title[nearest[k]]
        print(close_paper)

------ Nearest to  Semantic Approach for the Building of User Profile for Recommender System  -----------------
Parallel Multi-View Graph Matrix Completion for Large Input Matrix
The Current and Future Role of Smart Street Furniture in Smart Cities
Variance based product recommendation using clustering and sentiment analysis
Toward trustworthy social network services: A robust design of recommender systems
On output feedback controller design for Gaussian process state space models
Comparison of forecasting methods for power system short-term load forecasting based on neural networks
Threshold Certificate-Based Encryption: Definition and Concrete Construction
Modified Finite-Difference Time-Domain Method for Triangular Lattice Photonic Crystals
Non-myopic active learning with performance guarantee
Context aware personalized content recommendation using ontology based spreading activation


In [0]:
for i in range(len(testIndexes)):
    paper = embed2Title[testIndexes[i]]
    top_k = 10  # number of nearest neighbors
    nearest = (-simNumpyCombAfter[i, :]).argsort()[1:top_k + 1]
    print('------ Nearest to ', paper, ' -----------------')
    for k in range(top_k):
        close_paper = embed2Title[nearest[k]]
        print(close_paper)

------ Nearest to  Semantic Approach for the Building of User Profile for Recommender System  -----------------
Recommender Systems
Context aware personalized content recommendation using ontology based spreading activation
Ontology-based approach to provide personalized search results for handicraft woman
The Current and Future Role of Smart Street Furniture in Smart Cities
Parallel Multi-View Graph Matrix Completion for Large Input Matrix
Variance based product recommendation using clustering and sentiment analysis
Role of Ontology and Machine Learning in Recommender Systems
Toward trustworthy social network services: A robust design of recommender systems
Recommendation Engine for Online Short Video
Two-level matrix factorization for recommender systems


In [0]:
for i in range(len(testIndexes)):
    paper = embed2Title[testIndexes[i]]
    top_k = 10  # number of nearest neighbors
    nearest = (-simNumpyCombAfter[i, :]).argsort()[1:top_k + 1]
    print('------ Nearest to ', paper, ' -----------------')
    for k in range(top_k):
        close_paper = embed2Title[nearest[k]]
        print(close_paper)

------ Nearest to  Semantic Approach for the Building of User Profile for Recommender System  -----------------
Recommender Systems
Context aware personalized content recommendation using ontology based spreading activation
Ontology-based approach to provide personalized search results for handicraft woman
The Current and Future Role of Smart Street Furniture in Smart Cities
Parallel Multi-View Graph Matrix Completion for Large Input Matrix
Variance based product recommendation using clustering and sentiment analysis
Role of Ontology and Machine Learning in Recommender Systems
Toward trustworthy social network services: A robust design of recommender systems
Recommendation Engine for Online Short Video
Two-level matrix factorization for recommender systems
